In [1]:
import sqlite3
import pandas as pd

# 定义两个数据库文件路径
db_file1 = 'stock_data.db'
table_name = 'daily_stock_data'

# 读取第一个数据库
conn = sqlite3.connect(db_file1)
df1 = pd.read_sql(f'SELECT * FROM "{table_name}"', conn)
conn.close()



In [2]:
df1

,time,code,open,close,high,low,avg,volume,money,pre_close,market_cap,circulating_market_cap,turnover_ratio,pe_ratio,pb_ratio,ps_ratio,pcf_ratio
0,2014-01-02,000001.XSHE,5.38,5.43,5.46,5.35,5.40,110399265.0,5.962237e+08,5.44,1002.5372,681.9328,0.8786,6.7458,1.0563,2.1078,1.7328
1,2014-01-02,000002.XSHE,5.13,5.13,5.18,5.08,5.12,75587848.0,3.873911e+08,5.16,880.0883,773.1195,0.5015,6.4563,1.2892,0.7310,23.1340
2,2014-01-02,000063.XSHE,9.80,10.32,10.35,9.78,10.12,65024261.0,6.581458e+08,9.78,474.3807,386.4316,1.7373,-80.5540,2.1941,0.6071,131.0607
3,2014-01-02,000100.XSHE,1.69,1.68,1.69,1.67,1.68,105045800.0,1.762902e+08,1.69,197.2269,188.0401,0.9380,12.5332,1.5142,0.2398,10.8911
4,2014-01-02,000157.XSHE,3.63,3.61,3.63,3.59,3.61,34989562.0,1.262198e+08,3.65,415.3509,337.4943,0.3738,9.9725,0.9989,1.0971,-27.9564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462985,2025-07-17,601919.XSHG,15.44,15.17,15.47,15.16,15.22,96723274.0,1.472603e+09,15.39,2349.7958,1912.9272,0.7670,4.3483,0.9625,0.9649,17.3104
462986,2025-07-17,601939.XSHG,9.70,9.66,9.73,9.63,9.66,75020481.0,7.247082e+08,9.72,25270.5968,926.7473,0.7820,7.6091,0.7826,3.4182,-9.0720
462987,2025-07-17,601988.XSHG,5.71,5.67,5.73,5.66,5.68,176157307.0,1.001184e+09,5.71,18269.4437,11950.4047,0.0836,7.7342,0.7499,2.8807,-2.9844
462988,2025-07-17,601989.XSHG,4.60,4.65,4.65,4.58,4.61,122333347.0,5.639308e+08,4.59,1060.2946,1060.2946,0.5365,62.5921,1.2532,1.8446,3079.2957


In [28]:
import pandas as pd

# 1. 确保日期和股票代码排序正确
df1['time'] = pd.to_datetime(df1['time'])  # 转换为datetime格式
df1 = df1.sort_values(by=['code', 'time']).reset_index(drop=True)  # 按股票+日期排序

# 2. 按股票分组，计算未来5日的收盘价（shift(-5)）
df1['future_5day_close'] = df1.groupby('code')['close'].shift(-5)  # 未来5日收盘价

# 3. 计算未来5日累计收益率（百分比）
df1['future_5day_return'] = (
    (df1['future_5day_close'] - df1['close']) / df1['close'] * 100
)

# 4. 查看结果（示例：筛选某只股票的前10行）
sample_code = '000001.XSHE'
result = df1[df1['code'] == sample_code][
    ['time', 'code', 'close', 'future_5day_close', 'future_5day_return']
].head(10)

print(result)

        time         code  close  future_5day_close  future_5day_return
0 2014-01-02  000001.XSHE   5.43               5.25           -3.314917
1 2014-01-03  000001.XSHE   5.29               5.25           -0.756144
2 2014-01-06  000001.XSHE   5.18               5.15           -0.579151
3 2014-01-07  000001.XSHE   5.16               5.20            0.775194
4 2014-01-08  000001.XSHE   5.22               5.18           -0.766284
5 2014-01-09  000001.XSHE   5.25               5.19           -1.142857
6 2014-01-10  000001.XSHE   5.25               5.09           -3.047619
7 2014-01-13  000001.XSHE   5.15               5.01           -2.718447
8 2014-01-14  000001.XSHE   5.20               5.04           -3.076923
9 2014-01-15  000001.XSHE   5.18               5.23            0.965251


In [29]:
df1

,time,code,open,close,high,low,avg,volume,money,pre_close,market_cap,circulating_market_cap,turnover_ratio,pe_ratio,pb_ratio,ps_ratio,pcf_ratio,future_5day_close,future_5day_return
0,2014-01-02,000001.XSHE,5.38,5.43,5.46,5.35,5.40,110399265.0,5.962237e+08,5.44,1002.5372,681.9328,0.8786,6.7458,1.0563,2.1078,1.7328,5.25,-3.314917
1,2014-01-03,000001.XSHE,5.39,5.29,5.40,5.23,5.29,124191308.0,6.566313e+08,5.43,977.9451,665.2051,0.9884,6.5803,1.0304,2.0561,1.6903,5.25,-0.756144
2,2014-01-06,000001.XSHE,5.28,5.18,5.33,5.10,5.18,131177788.0,6.792804e+08,5.29,956.6320,650.7077,1.0440,6.4369,1.0079,2.0113,1.6535,5.15,-0.579151
3,2014-01-07,000001.XSHE,5.12,5.16,5.22,5.11,5.17,76258639.0,3.939776e+08,5.18,953.3530,648.4774,0.6069,6.4148,1.0044,2.0044,1.6478,5.20,0.775194
4,2014-01-08,000001.XSHE,5.17,5.22,5.30,5.12,5.22,103156044.0,5.384362e+08,5.16,1119.6397,655.7261,0.8210,7.5337,1.1796,2.3540,1.9352,5.18,-0.766284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462985,2025-07-11,601998.XSHG,8.77,8.51,8.81,8.51,8.66,86638078.0,7.503208e+08,8.74,4735.4033,3468.9312,0.2125,6.8735,0.6731,2.2374,5.2519,NaN,NaN
462986,2025-07-14,601998.XSHG,8.54,8.67,8.75,8.53,8.67,79682947.0,6.909424e+08,8.51,4824.4356,3534.1520,0.1955,7.0027,0.6858,2.2795,5.3507,NaN,NaN
462987,2025-07-15,601998.XSHG,8.66,8.61,8.72,8.54,8.63,62692182.0,5.412212e+08,8.67,4791.0485,3509.6942,0.1538,6.9542,0.6810,2.2637,5.3136,NaN,NaN
462988,2025-07-16,601998.XSHG,8.60,8.56,8.64,8.43,8.50,51579181.0,4.385205e+08,8.61,4763.2259,3489.3127,0.1265,6.9138,0.6771,2.2506,5.2828,NaN,NaN


In [30]:
# 只删除 future_5day_return 列存在空缺值的行
before = len(df1)
data = df1.dropna(subset=['future_5day_return']).reset_index(drop=True)  # 仅删除目标列有缺失的行
after = len(data)
print(f"\n仅删除 {'future_5day_return'} 列空缺值:")
print(f"删除前: {before} 行 → 删除后: {after} 行")
print(f"共删除 {before - after} 行目标值缺失的数据")


仅删除 future_5day_return 列空缺值:
删除前: 462990 行 → 删除后: 462165 行
共删除 825 行目标值缺失的数据


In [31]:
data

,time,code,open,close,high,low,avg,volume,money,pre_close,market_cap,circulating_market_cap,turnover_ratio,pe_ratio,pb_ratio,ps_ratio,pcf_ratio,future_5day_close,future_5day_return
0,2014-01-02,000001.XSHE,5.38,5.43,5.46,5.35,5.40,110399265.0,5.962237e+08,5.44,1002.5372,681.9328,0.8786,6.7458,1.0563,2.1078,1.7328,5.25,-3.314917
1,2014-01-03,000001.XSHE,5.39,5.29,5.40,5.23,5.29,124191308.0,6.566313e+08,5.43,977.9451,665.2051,0.9884,6.5803,1.0304,2.0561,1.6903,5.25,-0.756144
2,2014-01-06,000001.XSHE,5.28,5.18,5.33,5.10,5.18,131177788.0,6.792804e+08,5.29,956.6320,650.7077,1.0440,6.4369,1.0079,2.0113,1.6535,5.15,-0.579151
3,2014-01-07,000001.XSHE,5.12,5.16,5.22,5.11,5.17,76258639.0,3.939776e+08,5.18,953.3530,648.4774,0.6069,6.4148,1.0044,2.0044,1.6478,5.20,0.775194
4,2014-01-08,000001.XSHE,5.17,5.22,5.30,5.12,5.22,103156044.0,5.384362e+08,5.16,1119.6397,655.7261,0.8210,7.5337,1.1796,2.3540,1.9352,5.18,-0.766284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462160,2025-07-04,601998.XSHG,8.42,8.72,8.73,8.40,8.62,82423006.0,7.104346e+08,8.41,4946.8549,3623.8306,0.1984,7.1804,0.7032,2.3373,5.4864,8.51,-2.408257
462161,2025-07-07,601998.XSHG,8.72,8.81,8.84,8.63,8.74,57547060.0,5.029291e+08,8.72,4996.9356,3660.5173,0.1385,7.2531,0.7103,2.3610,5.5420,8.67,-1.589103
462162,2025-07-08,601998.XSHG,8.80,8.78,8.87,8.64,8.74,51612927.0,4.510224e+08,8.81,4980.2420,3648.2884,0.1242,7.2288,0.7079,2.3531,5.5235,8.61,-1.936219
462163,2025-07-09,601998.XSHG,8.77,8.79,8.90,8.69,8.83,55548005.0,4.902463e+08,8.78,4985.8065,3652.3647,0.1337,7.2369,0.7087,2.3557,5.5296,8.56,-2.616610


In [32]:
# 高效查找所有空缺值的行索引和列名
import numpy as np

# 使用 np.isnan 直接定位所有 NaN 的位置
nan_mask = data.isna().values  # 转为布尔型掩码
nan_indices = np.argwhere(nan_mask)  # 获取所有 True 的坐标

# 将坐标转换为行索引和列名
missing_values = []
for row_idx, col_idx in nan_indices:
    row_label = data.index[row_idx]  # 实际行索引
    col_name = data.columns[col_idx]  # 列名
    missing_values.append((row_label, col_name))

# 输出结果
print(f"数据中共有 {len(missing_values)} 个空缺值")
if missing_values:
    print("\n前20个空缺值的位置（行索引, 列名）：")
    for pos in missing_values[:20]:
        print(pos)

数据中共有 9885 个空缺值

前20个空缺值的位置（行索引, 列名）：
(129, 'turnover_ratio')
(3281, 'turnover_ratio')
(3282, 'turnover_ratio')
(3283, 'turnover_ratio')
(3284, 'turnover_ratio')
(3285, 'turnover_ratio')
(3286, 'turnover_ratio')
(3287, 'turnover_ratio')
(3288, 'turnover_ratio')
(3289, 'turnover_ratio')
(3290, 'turnover_ratio')
(3291, 'turnover_ratio')
(3292, 'turnover_ratio')
(3293, 'turnover_ratio')
(3294, 'turnover_ratio')
(3295, 'turnover_ratio')
(3296, 'turnover_ratio')
(3297, 'turnover_ratio')
(3298, 'turnover_ratio')
(3299, 'turnover_ratio')


In [33]:
# 确保时间列是 datetime 格式（用于正确排序）
data['time'] = pd.to_datetime(data['time'])

# 使用推荐方法替代弃用的 groupby().fillna()
data_filled = (
    data.sort_values(by=['code', 'time'])
    .groupby('code', group_keys=False)
    .apply(lambda x: x.ffill())  # 向上填充（前向填充）
    .fillna(0)  # 将剩余缺失值填充为0
)

# 检查填充效果
before = data.isna().sum().sum()  # 填充前总缺失值数量
after = data_filled.isna().sum().sum()  # 填充后总缺失值数量
print(f"缺失值填充完成：")
print(f"填充前总缺失值：{before} 个")
print(f"填充后总缺失值：{after} 个")
print(f"向上填充数量：{before - after} 个")

缺失值填充完成：
填充前总缺失值：9885 个
填充后总缺失值：0 个
向上填充数量：9885 个


C:\Users\Administrator\AppData\Local\Temp\ipykernel_37176\2383335253.py:8: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.ffill())  # 向上填充（前向填充）


In [35]:
# 按股票代码分组，统计每个股票的交易日数据量
stock_trading_days = data_filled.groupby('code').size().reset_index(name='交易日数量')

# 按交易日数量降序排列（可选）
stock_trading_days = stock_trading_days.sort_values(by='交易日数量', ascending=False)

stock_trading_days

,code,交易日数量
0,000001.XSHE,2801
104,600460.XSHG,2801
106,600489.XSHG,2801
107,600515.XSHG,2801
108,600519.XSHG,2801
...,...,...
56,300033.XSHE,2801
57,300059.XSHE,2801
58,600000.XSHG,2801
59,600009.XSHG,2801


In [36]:
stocks_data=data_filled
stocks_data

,time,code,open,close,high,low,avg,volume,money,pre_close,market_cap,circulating_market_cap,turnover_ratio,pe_ratio,pb_ratio,ps_ratio,pcf_ratio,future_5day_close,future_5day_return
0,2014-01-02,000001.XSHE,5.38,5.43,5.46,5.35,5.40,110399265.0,5.962237e+08,5.44,1002.5372,681.9328,0.8786,6.7458,1.0563,2.1078,1.7328,5.25,-3.314917
1,2014-01-03,000001.XSHE,5.39,5.29,5.40,5.23,5.29,124191308.0,6.566313e+08,5.43,977.9451,665.2051,0.9884,6.5803,1.0304,2.0561,1.6903,5.25,-0.756144
2,2014-01-06,000001.XSHE,5.28,5.18,5.33,5.10,5.18,131177788.0,6.792804e+08,5.29,956.6320,650.7077,1.0440,6.4369,1.0079,2.0113,1.6535,5.15,-0.579151
3,2014-01-07,000001.XSHE,5.12,5.16,5.22,5.11,5.17,76258639.0,3.939776e+08,5.18,953.3530,648.4774,0.6069,6.4148,1.0044,2.0044,1.6478,5.20,0.775194
4,2014-01-08,000001.XSHE,5.17,5.22,5.30,5.12,5.22,103156044.0,5.384362e+08,5.16,1119.6397,655.7261,0.8210,7.5337,1.1796,2.3540,1.9352,5.18,-0.766284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462160,2025-07-04,601998.XSHG,8.42,8.72,8.73,8.40,8.62,82423006.0,7.104346e+08,8.41,4946.8549,3623.8306,0.1984,7.1804,0.7032,2.3373,5.4864,8.51,-2.408257
462161,2025-07-07,601998.XSHG,8.72,8.81,8.84,8.63,8.74,57547060.0,5.029291e+08,8.72,4996.9356,3660.5173,0.1385,7.2531,0.7103,2.3610,5.5420,8.67,-1.589103
462162,2025-07-08,601998.XSHG,8.80,8.78,8.87,8.64,8.74,51612927.0,4.510224e+08,8.81,4980.2420,3648.2884,0.1242,7.2288,0.7079,2.3531,5.5235,8.61,-1.936219
462163,2025-07-09,601998.XSHG,8.77,8.79,8.90,8.69,8.83,55548005.0,4.902463e+08,8.78,4985.8065,3652.3647,0.1337,7.2369,0.7087,2.3557,5.5296,8.56,-2.616610


In [38]:
stocks_data.to_csv('stocks_data.csv', index=False)

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('stocks_data.csv')
df

,time,code,open,close,high,low,avg,volume,money,pre_close,market_cap,circulating_market_cap,turnover_ratio,pe_ratio,pb_ratio,ps_ratio,pcf_ratio,future_5day_close,future_5day_return
0,2014-01-02,000001.XSHE,5.38,5.43,5.46,5.35,5.40,110399265.0,5.962237e+08,5.44,1002.5372,681.9328,0.8786,6.7458,1.0563,2.1078,1.7328,5.25,-3.314917
1,2014-01-03,000001.XSHE,5.39,5.29,5.40,5.23,5.29,124191308.0,6.566313e+08,5.43,977.9451,665.2051,0.9884,6.5803,1.0304,2.0561,1.6903,5.25,-0.756144
2,2014-01-06,000001.XSHE,5.28,5.18,5.33,5.10,5.18,131177788.0,6.792804e+08,5.29,956.6320,650.7077,1.0440,6.4369,1.0079,2.0113,1.6535,5.15,-0.579151
3,2014-01-07,000001.XSHE,5.12,5.16,5.22,5.11,5.17,76258639.0,3.939776e+08,5.18,953.3530,648.4774,0.6069,6.4148,1.0044,2.0044,1.6478,5.20,0.775194
4,2014-01-08,000001.XSHE,5.17,5.22,5.30,5.12,5.22,103156044.0,5.384362e+08,5.16,1119.6397,655.7261,0.8210,7.5337,1.1796,2.3540,1.9352,5.18,-0.766284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462160,2025-07-04,601998.XSHG,8.42,8.72,8.73,8.40,8.62,82423006.0,7.104346e+08,8.41,4946.8549,3623.8306,0.1984,7.1804,0.7032,2.3373,5.4864,8.51,-2.408257
462161,2025-07-07,601998.XSHG,8.72,8.81,8.84,8.63,8.74,57547060.0,5.029291e+08,8.72,4996.9356,3660.5173,0.1385,7.2531,0.7103,2.3610,5.5420,8.67,-1.589103
462162,2025-07-08,601998.XSHG,8.80,8.78,8.87,8.64,8.74,51612927.0,4.510224e+08,8.81,4980.2420,3648.2884,0.1242,7.2288,0.7079,2.3531,5.5235,8.61,-1.936219
462163,2025-07-09,601998.XSHG,8.77,8.79,8.90,8.69,8.83,55548005.0,4.902463e+08,8.78,4985.8065,3652.3647,0.1337,7.2369,0.7087,2.3557,5.5296,8.56,-2.616610


In [2]:
stock_codes = df['code'].unique()
stock_list = list(stock_codes)
stock_list

['000001.XSHE',
 '000002.XSHE',
 '000063.XSHE',
 '000100.XSHE',
 '000157.XSHE',
 '000301.XSHE',
 '000338.XSHE',
 '000408.XSHE',
 '000425.XSHE',
 '000538.XSHE',
 '000568.XSHE',
 '000596.XSHE',
 '000617.XSHE',
 '000625.XSHE',
 '000630.XSHE',
 '000651.XSHE',
 '000661.XSHE',
 '000708.XSHE',
 '000725.XSHE',
 '000768.XSHE',
 '000776.XSHE',
 '000786.XSHE',
 '000792.XSHE',
 '000800.XSHE',
 '000807.XSHE',
 '000858.XSHE',
 '000876.XSHE',
 '000895.XSHE',
 '000938.XSHE',
 '000963.XSHE',
 '000975.XSHE',
 '000977.XSHE',
 '000983.XSHE',
 '000999.XSHE',
 '002001.XSHE',
 '002027.XSHE',
 '002028.XSHE',
 '002049.XSHE',
 '002050.XSHE',
 '002074.XSHE',
 '002129.XSHE',
 '002142.XSHE',
 '002179.XSHE',
 '002180.XSHE',
 '002230.XSHE',
 '002236.XSHE',
 '002241.XSHE',
 '002252.XSHE',
 '002304.XSHE',
 '002311.XSHE',
 '002352.XSHE',
 '002371.XSHE',
 '002415.XSHE',
 '002422.XSHE',
 '300014.XSHE',
 '300015.XSHE',
 '300033.XSHE',
 '300059.XSHE',
 '600000.XSHG',
 '600009.XSHG',
 '600010.XSHG',
 '600011.XSHG',
 '600015

In [2]:
# 选择特征和目标
features = [
    'open', 'close', 'high', 'low', 'avg', 'volume', 'money',
    'pre_close', 'market_cap', 'circulating_market_cap',
    'turnover_ratio', 'pe_ratio', 'pb_ratio', 'ps_ratio', 'pcf_ratio'
]
target = 'future_5day_return'

# 只保留需要的列，去除缺失
data = df[features + [target, 'code', 'time']].dropna().reset_index(drop=True)

# 按股票分组，构造训练样本（以时间为序）
groups = data.groupby('code')

# 构造训练集X, y（T, N）格式，T为时间，N为股票数
# 这里只做简单示例：用所有股票的同一天为一个截面
# 你可以根据实际需求调整
pivoted = {}
for col in features + [target]:
    pivoted[col] = data.pivot(index='time', columns='code', values=col)

# 训练集
X_dict = {f: pivoted[f].values for f in features}
y = pivoted[target].values  # (T, N)

In [3]:
X_dict 

{'open': array([[ 5.38,  5.13,  9.8 , ...,  1.37,  5.47,  2.29],
        [ 5.39,  5.12, 10.3 , ...,  1.36,  5.47,  2.28],
        [ 5.28,  5.02, 10.47, ...,  1.34,  5.38,  2.26],
        ...,
        [12.75,  6.51, 31.9 , ...,  5.72,  4.65,  8.8 ],
        [12.66,  6.54, 32.66, ...,  5.72,  4.6 ,  8.77],
        [12.82,  6.52, 32.86, ...,  5.79,  4.55,  8.83]]),
 'close': array([[ 5.43,  5.13, 10.32, ...,  1.36,  5.5 ,  2.29],
        [ 5.29,  5.03, 10.53, ...,  1.34,  5.4 ,  2.26],
        [ 5.18,  4.8 , 10.2 , ...,  1.32,  5.09,  2.22],
        ...,
        [12.69,  6.56, 32.72, ...,  5.71,  4.6 ,  8.78],
        [12.84,  6.54, 32.86, ...,  5.78,  4.54,  8.79],
        [13.18,  6.76, 33.12, ...,  5.82,  4.54,  8.74]]),
 'high': array([[ 5.46,  5.18, 10.35, ...,  1.37,  5.54,  2.31],
        [ 5.4 ,  5.12, 10.55, ...,  1.36,  5.48,  2.29],
        [ 5.33,  5.03, 10.47, ...,  1.34,  5.38,  2.26],
        ...,
        [12.84,  6.58, 32.78, ...,  5.75,  4.66,  8.87],
        [12.9 ,  6.5

In [7]:
y.shape

(2801, 165)

In [5]:
data

,open,close,high,low,avg,volume,money,pre_close,market_cap,circulating_market_cap,turnover_ratio,pe_ratio,pb_ratio,ps_ratio,pcf_ratio,future_5day_return,code,time
0,5.38,5.43,5.46,5.35,5.40,110399265.0,5.962237e+08,5.44,1002.5372,681.9328,0.8786,6.7458,1.0563,2.1078,1.7328,-3.314917,000001.XSHE,2014-01-02
1,5.39,5.29,5.40,5.23,5.29,124191308.0,6.566313e+08,5.43,977.9451,665.2051,0.9884,6.5803,1.0304,2.0561,1.6903,-0.756144,000001.XSHE,2014-01-03
2,5.28,5.18,5.33,5.10,5.18,131177788.0,6.792804e+08,5.29,956.6320,650.7077,1.0440,6.4369,1.0079,2.0113,1.6535,-0.579151,000001.XSHE,2014-01-06
3,5.12,5.16,5.22,5.11,5.17,76258639.0,3.939776e+08,5.18,953.3530,648.4774,0.6069,6.4148,1.0044,2.0044,1.6478,0.775194,000001.XSHE,2014-01-07
4,5.17,5.22,5.30,5.12,5.22,103156044.0,5.384362e+08,5.16,1119.6397,655.7261,0.8210,7.5337,1.1796,2.3540,1.9352,-0.766284,000001.XSHE,2014-01-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452275,8.42,8.72,8.73,8.40,8.62,82423006.0,7.104346e+08,8.41,4946.8549,3623.8306,0.1984,7.1804,0.7032,2.3373,5.4864,-2.408257,601998.XSHG,2025-07-04
452276,8.72,8.81,8.84,8.63,8.74,57547060.0,5.029291e+08,8.72,4996.9356,3660.5173,0.1385,7.2531,0.7103,2.3610,5.5420,-1.589103,601998.XSHG,2025-07-07
452277,8.80,8.78,8.87,8.64,8.74,51612927.0,4.510224e+08,8.81,4980.2420,3648.2884,0.1242,7.2288,0.7079,2.3531,5.5235,-1.936219,601998.XSHG,2025-07-08
452278,8.77,8.79,8.90,8.69,8.83,55548005.0,4.902463e+08,8.78,4985.8065,3652.3647,0.1337,7.2369,0.7087,2.3557,5.5296,-2.616610,601998.XSHG,2025-07-09


In [ ]:
# 选择特征和目标
features = [
    'open', 'close', 'high', 'low', 'avg', 'volume', 'money',
    'pre_close', 'market_cap', 'circulating_market_cap',
    'turnover_ratio', 'pe_ratio', 'pb_ratio', 'ps_ratio', 'pcf_ratio'
]
target = 'future_5day_return'

# 数据预处理
print(f"原始数据行数: {len(df)}")
data = df[features + [target, 'code', 'time']].copy()



# 构造训练集X, y（T, N）格式，T为时间，N为股票数
print("\n构造训练数据...")
pivoted = {}
for col in features + [target]:
    try:
        pivoted[col] = data.pivot(index='time', columns='code', values=col)
        print(f"特征 {col} 转换成功，形状: {pivoted[col].shape}")
    except Exception as e:
        print(f"特征 {col} 转换失败: {e}")
        # 创建全零矩阵作为替代
        unique_dates = data['time'].unique()
        unique_codes = data['code'].unique()
        pivoted[col] = pd.DataFrame(0, index=unique_dates, columns=unique_codes)

# 训练集
X_dict = {f: pivoted[f].values for f in features}
y = pivoted[target].values  # (T, N)
print(f"\n训练数据准备完成:")
print(f"时间点数量: {y.shape[0]}")
print(f"股票数量: {y.shape[1]}")


原始数据行数: 462165

缺失值统计:
Series([], dtype: int64)

构造训练数据...
特征 open 转换成功，形状: (2801, 165)
特征 close 转换成功，形状: (2801, 165)
特征 high 转换成功，形状: (2801, 165)
特征 low 转换成功，形状: (2801, 165)
特征 avg 转换成功，形状: (2801, 165)
特征 volume 转换成功，形状: (2801, 165)
特征 money 转换成功，形状: (2801, 165)
特征 pre_close 转换成功，形状: (2801, 165)
特征 market_cap 转换成功，形状: (2801, 165)
特征 circulating_market_cap 转换成功，形状: (2801, 165)
特征 turnover_ratio 转换成功，形状: (2801, 165)
特征 pe_ratio 转换成功，形状: (2801, 165)
特征 pb_ratio 转换成功，形状: (2801, 165)
特征 ps_ratio 转换成功，形状: (2801, 165)
特征 pcf_ratio 转换成功，形状: (2801, 165)
特征 future_5day_return 转换成功，形状: (2801, 165)

训练数据准备完成:
时间点数量: 2801
股票数量: 165


In [41]:
X_dict

{'open': array([[ 5.38,  5.13,  9.8 , ...,  1.37,  5.47,  2.29],
        [ 5.39,  5.12, 10.3 , ...,  1.36,  5.47,  2.28],
        [ 5.28,  5.02, 10.47, ...,  1.34,  5.38,  2.26],
        ...,
        [12.75,  6.51, 31.9 , ...,  5.72,  4.65,  8.8 ],
        [12.66,  6.54, 32.66, ...,  5.72,  4.6 ,  8.77],
        [12.82,  6.52, 32.86, ...,  5.79,  4.55,  8.83]]),
 'close': array([[ 5.43,  5.13, 10.32, ...,  1.36,  5.5 ,  2.29],
        [ 5.29,  5.03, 10.53, ...,  1.34,  5.4 ,  2.26],
        [ 5.18,  4.8 , 10.2 , ...,  1.32,  5.09,  2.22],
        ...,
        [12.69,  6.56, 32.72, ...,  5.71,  4.6 ,  8.78],
        [12.84,  6.54, 32.86, ...,  5.78,  4.54,  8.79],
        [13.18,  6.76, 33.12, ...,  5.82,  4.54,  8.74]]),
 'high': array([[ 5.46,  5.18, 10.35, ...,  1.37,  5.54,  2.31],
        [ 5.4 ,  5.12, 10.55, ...,  1.36,  5.48,  2.29],
        [ 5.33,  5.03, 10.47, ...,  1.34,  5.38,  2.26],
        ...,
        [12.84,  6.58, 32.78, ...,  5.75,  4.66,  8.87],
        [12.9 ,  6.5

In [42]:
# 验证每只股票的缺失值数量
missing_days = {}
for stock_code in pivoted['open'].columns:
    missing_count = pivoted['open'][stock_code].isna().sum()
    missing_days[stock_code] = missing_count
    
# 打印缺失值最多的10只股票
print("\n缺失交易日最多的10只股票:")
for code, count in sorted(missing_days.items(), key=lambda x: x[1], reverse=True)[:10]:
    actual_days = 2801 - count
    print(f"股票代码: {code}, 缺失天数: {count}, 实际交易天数: {actual_days}")


缺失交易日最多的10只股票:
股票代码: 000001.XSHE, 缺失天数: 0, 实际交易天数: 2801
股票代码: 000002.XSHE, 缺失天数: 0, 实际交易天数: 2801
股票代码: 000063.XSHE, 缺失天数: 0, 实际交易天数: 2801
股票代码: 000100.XSHE, 缺失天数: 0, 实际交易天数: 2801
股票代码: 000157.XSHE, 缺失天数: 0, 实际交易天数: 2801
股票代码: 000301.XSHE, 缺失天数: 0, 实际交易天数: 2801
股票代码: 000338.XSHE, 缺失天数: 0, 实际交易天数: 2801
股票代码: 000408.XSHE, 缺失天数: 0, 实际交易天数: 2801
股票代码: 000425.XSHE, 缺失天数: 0, 实际交易天数: 2801
股票代码: 000538.XSHE, 缺失天数: 0, 实际交易天数: 2801


In [43]:
y.shape

(2801, 165)

In [ ]:
import pandas as pd
import numpy as np
from gp_framework import GeneticProgrammer, FunctionSet
import time
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

# 读取数据
df = pd.read_csv('stocks_data.csv')

# 选择特征和目标
features = [
    'open', 'close', 'high', 'low', 'avg', 'volume', 'money',
    'pre_close', 'market_cap', 'circulating_market_cap',
    'turnover_ratio', 'pe_ratio', 'pb_ratio', 'ps_ratio', 'pcf_ratio'
]
target = 'future_5day_return'

# 数据预处理
print(f"原始数据行数: {len(df)}")
data = df[features + [target, 'code', 'time']].copy()



# 构造训练集X, y（T, N）格式，T为时间，N为股票数
print("\n构造训练数据...")
pivoted = {}
for col in features + [target]:
    try:
        pivoted[col] = data.pivot(index='time', columns='code', values=col)
        print(f"特征 {col} 转换成功，形状: {pivoted[col].shape}")
    except Exception as e:
        print(f"特征 {col} 转换失败: {e}")
        # 创建全零矩阵作为替代
        unique_dates = pd.Series(data['time']).unique()
        unique_codes = pd.Series(data['code']).unique()
        pivoted[col] = pd.DataFrame(0, index=unique_dates, columns=unique_codes)

# 训练集
X_dict = {f: pivoted[f].values for f in features}
y = pivoted[target].values  # (T, N)
print(f"\n训练数据准备完成:")
print(f"时间点数量: {y.shape[0]}")
print(f"股票数量: {y.shape[1]}")


def fitness_func(prog, X_dict, y, return_details=False):
    """
    计算因子表达式的适应度（时间截面rankIC均值）
    
    参数:
        prog: Node表达式树
        X_dict: dict of ndarray, 每个特征 (T, N)
        y: (T, N) 目标变量
        return_details: 是否返回详细信息（用于分析）
    
    返回:
        float: 所有时间截面rankIC的均值
    """
    try:
        # 计算因子值（直接使用传入的function_set实例，避免重复创建）
        pred = prog.evaluate(X_dict, FunctionSet())  # (T, N)
        
        # 确保pred是二维数组
        # 全量兜底，保证pred是float64二维数组
        pred = np.asarray(pred, dtype=np.float64)
        if pred.size == 1:
            pred = np.full_like(y, float(pred), dtype=np.float64)
        elif pred.shape == ():  # 0维
            pred = np.full_like(y, pred, dtype=np.float64)
        elif pred.ndim == 1:
            if pred.size == y.shape[0]:
                pred = np.tile(pred.reshape(-1, 1), (1, y.shape[1]))
            elif pred.size == y.shape[1]:
                pred = np.tile(pred.reshape(1, -1), (y.shape[0], 1))
            else:
                return -np.inf
        elif pred.shape != y.shape:
            try:
                pred = np.broadcast_to(pred, y.shape)
            except Exception:
                return -np.inf
        if pred.shape != y.shape:
            return -np.inf
        
        # 向量化计算每个时间截面的rankIC
        ic_values = []
        valid_ts = 0
        
        for t in range(y.shape[0]):
            # 当前时间截面的预测值和真实值
            pred_t = pred[t].ravel()
            y_t = y[t].ravel()
            
            # 过滤缺失值
            mask = ~np.isnan(pred_t) & ~np.isnan(y_t)
            if np.sum(mask) < 10:  # 至少需要10个有效样本
                continue
                
            # 计算Spearman秩相关系数（使用numpy实现，避免Series转换开销）
            pred_rank = pd.Series(pred_t[mask]).rank()
            y_rank = pd.Series(y_t[mask]).rank()
            corr = np.corrcoef(pred_rank, y_rank)[0, 1]
            
            if not np.isnan(corr):
                ic_values.append(corr)
                valid_ts += 1
        
        # 计算最终适应度（增加稳定性惩罚项）
        if len(ic_values) < 5:  # 至少需要5个有效时间点
            return -np.inf
            
        mean_ic = np.mean(ic_values)
        icir = mean_ic / (np.std(ic_values) + 1e-8)
        
        # 综合评分：IC均值 + ICIR（稳定性）
        fitness = mean_ic + 0.2 * np.tanh(icir)
        
        if return_details:
            return {
                'fitness': fitness,
                'mean_ic': mean_ic,
                'icir': icir,
                'valid_ts': valid_ts,
                'total_ts': y.shape[0]
            }
        
        return fitness
        
    except Exception as e:
        print(f"适应度计算错误: {e}")
        return -np.inf

# 优化后的遗传规划器配置
gp = GeneticProgrammer(
    generations=15,  # 增加迭代轮数
    population_size=50,  # 扩大种群规模
    n_components=5,  # 输出更多候选因子
    hall_of_fame=10,  # 增加精英保留数量
    function_set=FunctionSet(),
    variable_names=features,
    ts_window=20,  # 扩大时间窗口
    random_state=42,
    # 调整遗传操作概率
    p_crossover=0.7,  # 增加交叉概率
    p_subtree_mutation=0.2,
    p_hoist_mutation=0.05,
    p_point_mutation=0.05,
    # 增强复杂度控制
    parsimony_coefficient=0.002,  # 增加对复杂树的惩罚
    init_depth=(3, 8)  # 扩大初始树深度范围
)

# 改进的进度回调函数
def progress_callback(gen, avg_len, avg_fit, best_len, best_fit):
    """显示进化进度并估算剩余时间"""
    elapsed = time.time() - start_time
    if gen > 0:
        time_per_gen = elapsed / (gen + 1)
        time_left = (gp.generations - gen - 1) * time_per_gen
        time_str = f"{time_left/60:.2f}m" if time_left > 60 else f"{time_left:.2f}s"
    else:
        time_str = "?"
    print(f"Gen {gen:3d} | AvgLen: {avg_len:6.1f} | AvgFit: {avg_fit:7.4f} | "
          f"BestLen: {best_len:4d} | BestFit: {best_fit:7.4f} | Time: {time_str}")

# 训练
print('开始遗传规划因子挖掘...')
print('=' * 90)
print('|    Population    |           Best Individual            |       Factor Quality       |')
print('-' * 90)
print(' Gen   AvgLen    AvgFit    BestLen    BestFit     Time Left')
print('-' * 90)

start_time = time.time()
gp.fit(fitness_func=fitness_func, fitness_args=(X_dict, y), progress_callback=progress_callback)

# 输出最优因子及其绩效指标
print('\n最优因子表达式及其绩效:')
print('=' * 100)
for i, prog in enumerate(gp.best_programs_[:]):  # 只显示前3个因子
    details = fitness_func(prog, X_dict, y, return_details=True)
    expr = prog.to_str()
    depth = prog.depth()
    size = prog.size()
    print(f"因子 {i+1}:")
    print(f"  表达式: {expr}")
    print(f"  适应度: {details['fitness']:.6f} | 平均IC: {details['mean_ic']:.6f} | ICIR: {details['icir']:.6f}")
    print(f"  复杂度: 深度={depth}, 节点数={size}, 有效时间点={details['valid_ts']}/{details['total_ts']}")
    print('-' * 100)

原始数据行数: 462165

构造训练数据...
特征 open 转换成功，形状: (2801, 165)
特征 close 转换成功，形状: (2801, 165)
特征 high 转换成功，形状: (2801, 165)
特征 low 转换成功，形状: (2801, 165)
特征 avg 转换成功，形状: (2801, 165)
特征 volume 转换成功，形状: (2801, 165)
特征 money 转换成功，形状: (2801, 165)
特征 pre_close 转换成功，形状: (2801, 165)
特征 market_cap 转换成功，形状: (2801, 165)
特征 circulating_market_cap 转换成功，形状: (2801, 165)
特征 turnover_ratio 转换成功，形状: (2801, 165)
特征 pe_ratio 转换成功，形状: (2801, 165)
特征 pb_ratio 转换成功，形状: (2801, 165)
特征 ps_ratio 转换成功，形状: (2801, 165)
特征 pcf_ratio 转换成功，形状: (2801, 165)
特征 future_5day_return 转换成功，形状: (2801, 165)

训练数据准备完成:
时间点数量: 2801
股票数量: 165
开始遗传规划因子挖掘...
|    Population    |           Best Individual            |       Factor Quality       |
------------------------------------------------------------------------------------------
 Gen   AvgLen    AvgFit    BestLen    BestFit     Time Left
------------------------------------------------------------------------------------------
Gen   0 | AvgLen:   22.3 | AvgFit:    -inf | BestLen:    8 |

In [1]:
import pandas as pd
import numpy as np
from gp_framework import GeneticProgrammer, FunctionSet
import time
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

# 读取数据
df = pd.read_csv('stocks_data.csv')

# 选择特征和目标
features = [
    'open', 'close', 'high', 'low', 'avg', 'volume', 'money',
    'pre_close', 'market_cap', 'circulating_market_cap',
    'turnover_ratio', 'pe_ratio', 'pb_ratio', 'ps_ratio', 'pcf_ratio'
]
target = 'future_5day_return'

# 数据预处理
print(f"原始数据行数: {len(df)}")
data = df[features + [target, 'code', 'time']].copy()



# 构造训练集X, y（T, N）格式，T为时间，N为股票数
print("\n构造训练数据...")
pivoted = {}
for col in features + [target]:
    try:
        pivoted[col] = data.pivot(index='time', columns='code', values=col)
        print(f"特征 {col} 转换成功，形状: {pivoted[col].shape}")
    except Exception as e:
        print(f"特征 {col} 转换失败: {e}")
        # 创建全零矩阵作为替代
        unique_dates = pd.Series(data['time']).unique()
        unique_codes = pd.Series(data['code']).unique()
        pivoted[col] = pd.DataFrame(0, index=unique_dates, columns=unique_codes)

# 训练集
X_dict = {f: pivoted[f].values for f in features}
y = pivoted[target].values  # (T, N)
print(f"\n训练数据准备完成:")
print(f"时间点数量: {y.shape[0]}")
print(f"股票数量: {y.shape[1]}")


def fitness_func(prog, X_dict, y, return_details=False):
    """
    计算因子表达式的适应度（时间截面rankIC均值）
    
    参数:
        prog: Node表达式树
        X_dict: dict of ndarray, 每个特征 (T, N)
        y: (T, N) 目标变量
        return_details: 是否返回详细信息（用于分析）
    
    返回:
        float: 所有时间截面rankIC的均值
    """
    try:
        # 计算因子值（直接使用传入的function_set实例，避免重复创建）
        pred = prog.evaluate(X_dict, FunctionSet())  # (T, N)
        
        # 确保pred是二维数组
        # 全量兜底，保证pred是float64二维数组
        pred = np.asarray(pred, dtype=np.float64)
        # if pred.size == 1:
        #     pred = np.full_like(y, float(pred), dtype=np.float64)
        # elif pred.shape == ():  # 0维
        #     pred = np.full_like(y, pred, dtype=np.float64)
        # elif pred.ndim == 1:
        #     if pred.size == y.shape[0]:
        #         pred = np.tile(pred.reshape(-1, 1), (1, y.shape[1]))
        #     elif pred.size == y.shape[1]:
        #         pred = np.tile(pred.reshape(1, -1), (y.shape[0], 1))
        #     else:
        #         return -np.inf
        # elif pred.shape != y.shape:
        #     try:
        #         pred = np.broadcast_to(pred, y.shape)
        #     except Exception:
        #         return -np.inf
        if pred.shape != y.shape:
            return -np.inf
        
        # 向量化计算每个时间截面的rankIC
        ic_values = []
        valid_ts = 0
        
        for t in range(y.shape[0]):
            # 当前时间截面的预测值和真实值
            pred_t = pred[t].ravel()
            y_t = y[t].ravel()
            
            # 过滤缺失值
            mask = ~np.isnan(pred_t) & ~np.isnan(y_t)
            if np.sum(mask) < 10:  # 至少需要10个有效样本
                continue
                
            # 计算Spearman秩相关系数（使用numpy实现，避免Series转换开销）
            pred_rank = pd.Series(pred_t[mask]).rank()
            y_rank = pd.Series(y_t[mask]).rank()
            corr = np.corrcoef(pred_rank, y_rank)[0, 1]
            
            if not np.isnan(corr):
                ic_values.append(corr)
                valid_ts += 1
        
        # 计算最终适应度（增加稳定性惩罚项）
        if len(ic_values) < 5:  # 至少需要5个有效时间点
            return -np.inf
            
        mean_ic = np.mean(ic_values)
        icir = mean_ic / (np.std(ic_values) + 1e-8)
        
        # 综合评分：IC均值 + ICIR（稳定性）
        fitness = mean_ic + 0.2 * np.tanh(icir)
        
        if return_details:
            return {
                'fitness': fitness,
                'mean_ic': mean_ic,
                'icir': icir,
                'valid_ts': valid_ts,
                'total_ts': y.shape[0]
            }
        
        return fitness
        
    except Exception as e:
        print(f"适应度计算错误: {e}")
        return -np.inf

# 优化后的遗传规划器配置
gp = GeneticProgrammer(
    generations=7,  # 增加迭代轮数
    population_size=50,  # 扩大种群规模
    tournament_size=10,  # 添加这个：每次选5个个体进行锦标赛
    n_components=5,  # 输出更多候选因子
    hall_of_fame=7,  # 增加精英保留数量
    function_set=FunctionSet(),
    variable_names=features,
    ts_window=15,  # d 时间窗口
    random_state=30,
    const_range=None,  # 禁止常数使用
    p_crossover=0.7,
    p_subtree_mutation=0.2,
    p_hoist_mutation=0.05,
    p_point_mutation=0.05,
    # 增强复杂度控制
    parsimony_coefficient=0,  # 增加对复杂树的惩罚
    init_depth=(4, 7)  # 扩大初始树深度范围
)

# 训练
gp.fit(fitness_func=fitness_func, fitness_args=(X_dict, y))

# 输出最优因子及其绩效指标
print('\n最优因子表达式及其绩效:')
print('=' * 100)

def export_graphviz(node, dot=None, parent=None, node_id=None, counter=[0]):
    import graphviz
    if dot is None:
        dot = graphviz.Digraph(format='png')
        dot.attr('node', shape='ellipse', style='filled', color='lightblue')
        counter[0] = 0  # reset counter
    if node_id is None:
        node_id = f'n{counter[0]}'
        counter[0] += 1
    label = node.name if node.value is None else str(node.value)
    dot.node(node_id, label=label)
    for i, child in enumerate(node.children):
        child_id = f'n{counter[0]}'
        counter[0] += 1
        export_graphviz(child, dot, node_id, child_id, counter)
        dot.edge(node_id, child_id)
    return dot

for i, prog in enumerate(gp.best_programs_[:]):  # 显示所有因子
    details = fitness_func(prog, X_dict, y, return_details=True)
    expr = prog.to_str()
    depth = prog.depth()
    size = prog.size()
    print(f"因子 {i+1}:")
    print(f"  表达式: {expr}")
    print(f"  适应度: {details['fitness']:.6f} | 平均IC: {details['mean_ic']:.6f} | ICIR: {details['icir']:.6f}")
    print(f"  复杂度: 深度={depth}, 节点数={size}, 有效时间点={details['valid_ts']}/{details['total_ts']}")
    
    # 生成 graphviz 可视化图片
    try:
        dot = export_graphviz(prog)
        filename = f'factor_{i+1}_graphviz'
        dot.render(filename, format='png', cleanup=True)
        print(f"  Graphviz可视化: 已保存至 {filename}.png")
    except ImportError:
        print(f"  Graphviz可视化: 失败（需要安装 graphviz 包和系统依赖）")
    
    print('-' * 100)

原始数据行数: 462165

构造训练数据...
特征 open 转换成功，形状: (2801, 165)
特征 close 转换成功，形状: (2801, 165)
特征 high 转换成功，形状: (2801, 165)
特征 low 转换成功，形状: (2801, 165)
特征 avg 转换成功，形状: (2801, 165)
特征 volume 转换成功，形状: (2801, 165)
特征 money 转换成功，形状: (2801, 165)
特征 pre_close 转换成功，形状: (2801, 165)
特征 market_cap 转换成功，形状: (2801, 165)
特征 circulating_market_cap 转换成功，形状: (2801, 165)
特征 turnover_ratio 转换成功，形状: (2801, 165)
特征 pe_ratio 转换成功，形状: (2801, 165)
特征 pb_ratio 转换成功，形状: (2801, 165)
特征 ps_ratio 转换成功，形状: (2801, 165)
特征 pcf_ratio 转换成功，形状: (2801, 165)
特征 future_5day_return 转换成功，形状: (2801, 165)

训练数据准备完成:
时间点数量: 2801
股票数量: 165
开始遗传规划因子挖掘...
|    Population    |           Best Individual            |       Factor Quality       |
------------------------------------------------------------------------------------------
 Gen   AvgLen    AvgFit    BestLen    BestFit     Time Left
------------------------------------------------------------------------------------------
Gen   0 | AvgLen:   13.2 | AvgFit:    -inf | BestLen:   13 |